In [1]:
import random
import os
import re
import graphviz
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import datetime
import functools
import wandb

In [2]:
sns.set_theme()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")
# setting the wandb API from Kaggle secrets to avoid manual login
os.environ["WANDB_API_KEY"] = secret_value_0

In [4]:
api = wandb.Api()
artifact = api.artifact('ml-major-project-g3/major-project/covid19-uk-processed-dataset:latest')
artifact_dir = artifact.download()

In [5]:
!ls artifacts/covid19-uk-processed-dataset:v1

uk_processed.csv


In [6]:
# Drop all the *_Flag features like C5_Flag
# they encode regionwise specificity of the stringency
# measures, we shall ignore them for now
def drop_all_flags(df, cfg, first_time=True):
    temp = df.columns.tolist()
    cols = [i for i in temp if i.endswith('Flag')]
    df.drop(cols, axis=1, inplace=True)
    if first_time:
        cfg.dropped_features.extend(cols)
        cfg.preprocessing_fns.append(drop_all_flags)
    return df

In [7]:
# Drop the IndicesForDisplay
# for our data they are identical
# to the corresponding indices
# Dropping StringencyLegacyIndex, stringency_index
# as well
def drop_all_indices_display(df, cfg, first_time=True):
    temp = df.columns.tolist()
    cols = [i for i in temp if i.endswith('ForDisplay')]
    cols.append('StringencyLegacyIndex')
    cols.append('stringency_index')
    df.drop(cols, axis=1, inplace=True)
    if first_time:
        cfg.dropped_features.extend(cols)
        cfg.preprocessing_fns.append(drop_all_indices_display)
    return df

In [8]:
# drop all *_per_million , *_per_hundred and *_per_thousand features
def drop_all_per_features(df, cfg, first_time=True):
    temp = df.columns.tolist()
    cols = [i for i in temp if (i.endswith('_per_million') or
                                i.endswith('_per_hundred') or
                                i.endswith('_per_thousand'))]
    # dont remove this now, its a constant feature
    cols.remove('hospital_beds_per_thousand')
    df.drop(cols, axis=1, inplace=True)
    if first_time:
        cfg.dropped_features.extend(cols)
        cfg.preprocessing_fns.append(drop_all_per_features)
    return df

In [9]:
# dropping features that are identical
# in all datapoints, and store their
# values in a dict for later use
def drop_all_constant_features(df, cfg, first_time=True):
    cols = ['population', 'population_density', 'median_age', 'aged_65_older',
            'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
            'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
            'male_smokers', 'life_expectancy', 'human_development_index',
            'hospital_beds_per_thousand']

    if first_time:
        if cfg.constant_features is not None:
            for i in cols:
                cfg.constant_features[i] = df.head(1)[i].squeeze()
            cfg.constant_features['hospital_beds'] = (cfg.constant_features['hospital_beds_per_thousand'] *
                                                       cfg.constant_features['population'] / 1000)
            cfg.constant_features.pop('hospital_beds_per_thousand')
        cfg.dropped_features.extend(cols)
        cfg.preprocessing_fns.append(drop_all_constant_features)
    df.drop(cols, axis=1, inplace=True)
    return df

## Dropping all the `E_*` features

- 'E1_Income support', 'E2_Debt/contract relief' are constant throughout
- 'E3_Fiscal measures', 'E4_International support' are mostly 0, except for very few large peaks as the govt allocates a large amount of money in one go

These are unlikely to be useful

In [10]:
# dropping all the E_* features
# 'E1_Income support', 'E2_Debt/contract relief' are 
# but mostly a hunch

def drop_all_E_features(df, cfg, first_time=True):
    temp = df.columns.tolist()
    cols = ['E1_Income support', 'E2_Debt/contract relief',
            'E3_Fiscal measures', 'E4_International support']
    df.drop(cols, axis=1, inplace=True)
    if first_time:
        cfg.dropped_features.extend(cols)
        cfg.preprocessing_fns.append(drop_all_E_features)
    return df

## Feature Engineering


In [11]:
def add_engineered_features(df, cfg, first_time=True):
    df['susceptible_population'] = cfg.constant_features['population'] - df['people_vaccinated'] - df['total_cases']
    df['icu_fatality'] = df['new_deaths_smoothed']/df['icu_patients']
    df['icu_hosp_ratio'] = df['icu_patients']/df['hosp_patients']
    df['hosp_beds_left'] = cfg.constant_features['hospital_beds'] - df['hosp_patients']
    df['potential_cases'] = df['susceptible_population'] * df['positive_rate']
    df.fillna(value={
        'icu_fatality':0,
        'icu_hosp_ratio':0
    }, inplace=True)
    if first_time:
        cfg.preprocessing_fns.append(add_engineered_features)
    return df

In [12]:
# just trying what works

def drop_additional_features(df, cfg, first_time=True):
    temp = df.columns.tolist()
    cols = ['new_cases', 'new_deaths',
       'new_tests', 'total_tests', 'new_tests_smoothed',
       'tests_per_case', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6_Facial Coverings',
       'H7_Vaccination policy', 'C6_Stay at home requirements',
       'StringencyIndex', 'GovernmentResponseIndex', 'ContainmentHealthIndex',
       'EconomicSupportIndex']
    df.drop(cols, axis=1, inplace=True)
    if first_time:
        cfg.dropped_features.extend(cols)
        cfg.preprocessing_fns.append(drop_additional_features)
    return df

## Feature Scaling

Decision Trees are not affected by outliers due to the nature of the algorithm hence we really dont need to do any feature scaling.

## Converting to a Forecasting Problem

We will be trying to forecast deaths(actually `cfg.pred_var`), given the past.

At the moment each row contains the all the features for a particular date. What we are aiming to do with our model is to predict the future day(s) given the past. There are multiple ways of casting this to a forecasting problem, we choose the following way: Given `num_past_days` for the past we will try to predict the `num_future_days` of the future (multi-target prediction).

So each row of our new dataset will consist of all the feature values for `num_past_days` days and we will be trying to predict the deaths(`cfg.pred_var`) on the next `num_future_days` days.

Now we shall convert our dataset into the above form.

In [13]:
# setting the dates as the index
def set_dates_as_index(df, cfg, first_time=True):
    df.set_index('date', inplace=True, drop=True)
    df.index = pd.to_datetime(df.index)
    if first_time:
        cfg.preprocessing_fns.append(set_dates_as_index)
    return df


In [14]:
# We shall shift and join the dataset
# to get our required dataset
def shift_and_join(df, cfg, first_time=True):
    temp_df = df.copy()
    # shifting and joining past days data
    for i in range(1, cfg.num_past_days):
        temp_df = temp_df.join(df.shift(freq=datetime.timedelta(days=i)), rsuffix='_'+str(i)+'_p')
    # shifting an joining future pred_var
    for i in range(1, cfg.num_future_days+1):
        temp_df = temp_df.join((df[[cfg.pred_var]]).shift(freq=datetime.timedelta(days=-i)), rsuffix='_'+str(i)+'_f')
    # suffix _p_0 to all the columns of the present
    present_cols = df.columns.tolist()
    new_present_cols = [i+'_0_p' for i in present_cols]
    temp_df.rename(columns = dict(zip(present_cols, new_present_cols)), inplace=True)
    # get rid of days where we dont have past data
    temp_df.dropna(inplace=True)
    if first_time:
        cfg.preprocessing_fns.append(shift_and_join)
    return temp_df


In [15]:
def drop_immediate_past(df, cfg, first_time=True):
    past_days_to_drop = cfg.past_days_to_drop
    imm_past_days_to_keep = cfg.imm_past_days_to_keep
    if imm_past_days_to_keep <= past_days_to_drop:
        for i in range(imm_past_days_to_keep, past_days_to_drop):
            cols = [f for f in df.columns if f.endswith('_'+str(i)+'_p') and
                   not f == cfg.pred_var +'_'+str(i)+'_p']
            df.drop(cols, axis=1, inplace=True)
    if first_time:
        cfg.preprocessing_fns.append(drop_immediate_past)
    return df


## Feature Selection

Now all the feature names of the past are suffixed with `_p` and the the feature names of the future are suffixed with `_f`

We shall now split the train dataset into features(X) and targets(Y).

In [16]:
from sklearn.model_selection import train_test_split

def split_df_to_X_Y(df):
    X = df[[i for i in df.columns if i.endswith('_p')]].copy()
    Y = df[[i for i in df.columns if i.endswith('_f')]].copy()
    return X, Y

# Decision Tree Model

In [17]:
from sklearn.tree import DecisionTreeRegressor

## Hyperparameter Tuning



In [18]:
def sweep_fn():
    # just defaults, they will automatically
    # be replaced with the values given by the sweep agent
    config_defaults = {
        'max_depth': 3,
        'criterion': 'mse',
        'past_days_to_drop': 0,
        'imm_past_days_to_keep': 0,
        'min_samples_split': 2,
        
    }
    run = wandb.init(config=config_defaults, save_code=False)
    run.use_artifact('ml-major-project-g3/major-project/covid19-uk-processed-dataset:latest', type='dataset')
    
    # config common vars
    cfg = wandb.config
    cfg.TEST_START_DATE = '2021-01-01'
    cfg.TEST_END_DATE = '2021-04-06'
    cfg.train_val_split = 0.15
    cfg.drop_initial_datapoints = True
    cfg.dropped_features = []
    cfg.RANDOM_SEED = 42
    cfg.num_future_days = 14
    cfg.num_past_days = 30
    cfg.pred_var = 'new_deaths_smoothed'
    cfg.constant_features = dict()
    cfg.preprocessing_fns = []
    
    # reproducibility
    np.random.seed(cfg.RANDOM_SEED)
    random.seed(cfg.RANDOM_SEED)
    
    # load data
    df = pd.read_csv('./artifacts/covid19-uk-processed-dataset:v1/uk_processed.csv')
    df.sort_values('date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    # preprocessing
    if cfg.drop_initial_datapoints == True:
        df.dropna(inplace=True)
    start_idx = df[df['date'] == cfg.TEST_START_DATE].index[0]
    end_idx = df[df['date'] == cfg.TEST_END_DATE].index[0]
    train_df = df.loc[0:start_idx-1].copy()
    
    train_df = drop_all_flags(train_df, cfg)
    train_df = drop_all_indices_display(train_df, cfg)
    train_df = drop_all_per_features(train_df, cfg)
    train_df = drop_all_constant_features(train_df, cfg)
    train_df = drop_all_E_features(train_df, cfg)
    train_df = add_engineered_features(train_df, cfg)
    train_df = drop_additional_features(train_df, cfg)
    train_df = set_dates_as_index(train_df, cfg)
    train_df = shift_and_join(train_df, cfg)
    train_df = drop_immediate_past(train_df, cfg)
    # train - validation split
    train_X, train_Y = split_df_to_X_Y(train_df)
    train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size=cfg.train_val_split, shuffle=False)
    val_X = val_X.iloc[cfg.num_future_days:].copy()
    val_Y = val_Y.iloc[cfg.num_future_days:].copy()
    # model
    model = DecisionTreeRegressor(criterion=cfg.criterion,
                                  max_depth=cfg.max_depth,
                                  min_samples_split=cfg.min_samples_split,
                                  random_state=cfg.RANDOM_SEED,)
    model.fit(train_X, train_Y)
    
    def get_metrics(model, X_df, Y_df):
        preds = model.predict(X_df)
        errs = (preds - Y_df)
        mae_errs = abs(errs).mean(axis=0)
        mape_errs = (abs(errs)/Y_df).mean(axis=0)
        mse_errs = (errs**2).mean(axis=0)
        net_mse_errs = mse_errs.mean()
        net_mae_errs = mae_errs.mean()
        net_mape_errs = mape_errs.mean()
        return net_mse_errs, net_mae_errs, net_mape_errs
    # train metrics
    train_mse_err, train_mae_err, train_mape_err = get_metrics(model, train_X, train_Y)
    # validation metrics
    val_mse_err, val_mae_err, val_mape_err = get_metrics(model, val_X, val_Y)
    
    wandb.log({
        'train_mse': train_mse_err,
        'train_mae': train_mae_err,
        'train_mape': train_mape_err,
        'val_mse': val_mse_err,
        'val_mae': val_mae_err,
        'val_mape': val_mape_err,
    })
    

In [19]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'max_depth': {
            'values': [3, 4, 5, 6, 7],
        },
        'criterion': {
            'values': ['mse', 'friedman_mse', 'mae'],
        },
        'past_days_to_drop': {
            'values': [7, 14, 18]
        },
        'imm_past_days_to_keep': {
            'values': [3, 5, 7]
        },
        'min_samples_split': {
            'values': [2, 4, 8, 16, 32]
        },
    }
}

In [20]:
sweep_id = wandb.sweep(sweep_config, entity='ml-major-project-g3',
                       project='decision-tree-tuning')
wandb.agent(sweep_id, function=sweep_fn)

Create sweep with ID: dtuw5oy7
Sweep URL: https://wandb.ai/ml-major-project-g3/decision-tree-tuning/sweeps/dtuw5oy7


wandb: Agent Starting Run: el77vbfn with config:
wandb: 	criterion: mse
wandb: 	imm_past_days_to_keep: 3
wandb: 	max_depth: 3
wandb: 	min_samples_split: 2
wandb: 	past_days_to_drop: 7
wandb: Currently logged in as: enigma0160 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


train_mse,3372.06552
train_mae,41.69543
train_mape,inf
val_mse,14944.95883
val_mae,118.49583
val_mape,0.26345
_runtime,7
_timestamp,1619783217
_step,0


train_mse,▁
train_mae,▁
train_mape,
val_mse,▁
val_mae,▁
val_mape,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: oe143zkb with config:
wandb: 	criterion: mse
wandb: 	imm_past_days_to_keep: 3
wandb: 	max_depth: 3
wandb: 	min_samples_split: 2
wandb: 	past_days_to_drop: 14
wandb: wandb version 0.10.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

train_mse,▁
train_mae,▁
train_mape,
val_mse,▁
val_mae,▁
val_mape,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: 6vf73kv4 with config:
wandb: 	criterion: mae
wandb: 	imm_past_days_to_keep: 7
wandb: 	max_depth: 7
wandb: 	min_samples_split: 32
wandb: 	past_days_to_drop: 14
wandb: wandb version 0.10.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


train_mse,6508.21301
train_mae,36.55723
train_mape,inf
val_mse,30180.95814
val_mae,161.46838
val_mape,0.35779
_runtime,7
_timestamp,1619785730
_step,0


train_mse,▁
train_mae,▁
train_mape,
val_mse,▁
val_mae,▁
val_mape,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
